# Tutorial 7: Work with Groups

This Tutorial demonstrates how reassign element to differnt groups and create new elements and directley assign them to groups. For more advanced group operations see:

## SIR 3S Installation

In [1]:
SIR3S_SIRGRAF_DIR = r"C:\3S\SIR 3S\SirGraf-90-15-00-21_Quebec-Upd2" #change to local path

## Imports

Note: The SIR 3S Toolkit requires the Sir3S_Toolkit.dll included in SIR 3S installations (version Quebec and higher).

In [2]:
import sir3stoolkit

The core of sir3stoolkit is a Python wrapper around basic functionality of SIR 3S, offering a low-level access to the creation, modification and simulation of SIR 3S models. In the future pure python subpackages may be added.

In [3]:
from sir3stoolkit.core import wrapper

In [4]:
sir3stoolkit

<module 'sir3stoolkit' from 'C:\\Users\\aUsername\\3S\\sir3stoolkit\\src\\sir3stoolkit\\__init__.py'>

The [wrapper package](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.Initialize_Toolkit) has to be initialized with reference to a SIR 3S (SirGraf) installation.

In [5]:
wrapper.Initialize_Toolkit(SIR3S_SIRGRAF_DIR)

## Initialization

The SIR 3S Toolkit contains two classes: [SIR3S_Model](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model) (model and data) and [SIR3S_View](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_View) (depiction in SIR Graf). All SIR 3S Toolkit functionality is accessed via the methods of these classes.

In [6]:
s3s = wrapper.SIR3S_Model()

Initialization complete


In [7]:
s3s_view = wrapper.SIR3S_View()

Initialization complete


# Open Model

In [8]:
dbFilePath=r"Toolkit_Tutorial7_Model.db3"

In [9]:
s3s.OpenModel(dbName=dbFilePath,
              providerType=s3s.ProviderTypes.SQLite,
              Mid="M-1-0-1",
              saveCurrentlyOpenModel=False,
              namedInstance="",
              userID="",
              password="")

Model is open for further operation


In [10]:
print(s3s.GetNetworkType()) # to check that the correct model is responsive, model we are trying to open was created with type Undefined

NetworkType.Water


# View existing Groups and their content

## Finding and accessing groups

In [11]:
group_tks=s3s.GetTksofElementType(s3s.ObjectTypes.LAYR_Layer)

In [12]:
props = s3s.GetPropertiesofElementType(s3s.ObjectTypes.LAYR_Layer)

In [13]:
props

['Name',
 'Zeigen',
 'Setzen',
 'Lfdnr',
 'Idreferenz',
 'ObjsString',
 'Tk',
 'Pk',
 'InVariant']

In [14]:
for tk in group_tks:
        print(f"Tk: {tk}, Name: {s3s.GetValue(tk, props[0])[0]}, ObjsString: {s3s.GetValue(tk, props[5])[0]}")

Tk: 4999103085880923178, Name: Vorlauf, ObjsString: KNOT~4968737254281413660	KNOT~5130290893660176749	ROHR~5316743965501398998	KNOT~4906600779019373637	
Tk: 5238820536187712277, Name: Rücklauf, ObjsString: KNOT~5470637040092884723	ROHR~5736978569585339616	


We can see that this model only has the groups Vorlauf (Supply) and Rücklauf (Return). Both containing nodes and pipes.

## ObjsString

As we can see the ObjsString has an odd format. We will turn it into a python list.

In [15]:
supply_object_string = s3s.GetValue(group_tks[0], "ObjsString")[0]

In [16]:
supply_object_string

'KNOT~4968737254281413660\tKNOT~5130290893660176749\tROHR~5316743965501398998\tKNOT~4906600779019373637\t'

In [17]:
type(supply_object_string)

str

In [18]:
return_object_string = s3s.GetValue(group_tks[1], "ObjsString")[0]

In [19]:
return_object_string

'KNOT~5470637040092884723\tROHR~5736978569585339616\t'

### Change format

In [20]:
supply_elements_tks = []
for element in supply_object_string.split('\t'):
    if element: 
        obj_type, obj_id = element.split('~')
        supply_elements_tks.append((obj_type, obj_id))


In [21]:
supply_elements_tks

[('KNOT', '4968737254281413660'),
 ('KNOT', '5130290893660176749'),
 ('ROHR', '5316743965501398998'),
 ('KNOT', '4906600779019373637')]

Now we have a list of tuples with the element type and the tk of the object. We keep the element type, because if we want to insert of the elements into another group, we need to know it. With this we could continue into more complex python operations, but that is out of scope of this tutorial.

# Edit 

In this model there is a defect, the last node in the supply group actually belongs to the return lets fix this.

In [22]:
supply_object_string

'KNOT~4968737254281413660\tKNOT~5130290893660176749\tROHR~5316743965501398998\tKNOT~4906600779019373637\t'

In [23]:
return_object_string

'KNOT~5470637040092884723\tROHR~5736978569585339616\t'

For such short groups and simple operations it is the easiest to just reset the manually edited object strings.

In [24]:
supply_object_string = 'KNOT~4968737254281413660\tKNOT~5130290893660176749\tROHR~5316743965501398998\t'

In [25]:
return_object_string = 'KNOT~5470637040092884723\tROHR~5736978569585339616\tKNOT~4906600779019373637\t'

In [26]:
s3s.SetValue(group_tks[0], "ObjsString", supply_object_string)

Value is set


In [27]:
s3s.SetValue(group_tks[1], "ObjsString", return_object_string)

Value is set


# Insert newly added elements into groups

If we want to determine the groups elements will be inserted into when newly edited, we can set the attribute "Setzen" of the groups, we want them to be added to, to 1.

## Add a node to Vorlauf (supply) group

In [28]:
groups_to_add_to = group_tks[0] # Vorlauf (supply) group

In [29]:
for tk in group_tks:
    if tk in groups_to_add_to:
        s3s.SetValue(tk, "Setzen", "1")
    else:
        s3s.SetValue(tk, "Setzen", "0")

Value is set
Value is set


In [30]:
s3s.InsertElement(ElementType=s3s.ObjectTypes.Node, IdRef="-1") # Newly create element

Element inserted successfully into the model with Tk: 4919077416104559424


'4919077416104559424'

In [31]:
for tk in group_tks:
        print(f"Tk: {tk}, Name: {s3s.GetValue(tk, props[0])[0]}, ObjsString: {s3s.GetValue(tk, props[5])[0]}")

Tk: 4999103085880923178, Name: Vorlauf, ObjsString: KNOT~4968737254281413660	KNOT~5130290893660176749	ROHR~5316743965501398998	KNOT~4919077416104559424	
Tk: 5238820536187712277, Name: Rücklauf, ObjsString: KNOT~5470637040092884723	ROHR~5736978569585339616	KNOT~4906600779019373637	


## Add a node to Vorlauf (supply) and Rücklauf (return) group

In [32]:
groups_to_add_to = group_tks # Vorlauf (supply) and Rücklauf (return) group

In [33]:
for tk in group_tks:
    if tk in groups_to_add_to:
        s3s.SetValue(tk, "Setzen", "1")
    else:
        s3s.SetValue(tk, "Setzen", "0")

Value is set
Value is set


In [34]:
s3s.InsertElement(ElementType=s3s.ObjectTypes.Node, IdRef="-1") # Newly create element

Element inserted successfully into the model with Tk: 4695903655171808515


'4695903655171808515'

In [35]:
for tk in group_tks:
        print(f"Tk: {tk}, Name: {s3s.GetValue(tk, props[0])[0]}, ObjsString: {s3s.GetValue(tk, props[5])[0]}")

Tk: 4999103085880923178, Name: Vorlauf, ObjsString: KNOT~4968737254281413660	KNOT~5130290893660176749	ROHR~5316743965501398998	KNOT~4919077416104559424	KNOT~4695903655171808515	
Tk: 5238820536187712277, Name: Rücklauf, ObjsString: KNOT~5470637040092884723	ROHR~5736978569585339616	KNOT~4906600779019373637	KNOT~4695903655171808515	


In [36]:
s3s.CloseModel(False)

True